In [1]:
# to prevent to restart kernel when any changes are made to any imported file
%reload_ext autoreload
%autoreload 2

# to import any file from some other directory

# to stop printing warnings
import warnings
warnings.filterwarnings('ignore')
def warn(*args, **kwargs):
    pass
warnings.warn = warn
    
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 20000)
pd.set_option('display.max_colwidth', -1)

# to increase cells width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# to enable collapsible Headings and Functions
# !pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install --user
# !jupyter nbextensions_configurator enable --user
# !jupyter nbextension enable codefolding/main
# search collapsible to enable

# enable dark theme
# !pip install jupyterthemes
# !jt -t monokai

# monokai
# solarizedd
# !jt -r
import numpy as np
import pandas as pd
import sys, os
from datetime import datetime, timedelta
import requests
import json
import timeit
from datetime import datetime
import os.path, time
from datetime import datetime, timedelta
from threading import Timer
import pdb, random

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import preprocessing
from pprint import pprint




In [3]:
PATH="/home/cloud_user/faisal/experiments/real_not_real_kaggle/data1/"

In [ ]:
def evaluate(X_trn, y_trn,X_ts, y_ts, base_model):
    base_oob     = base_model.oob_score_
    base_acc_trn = base_model.score(X_trn, y_trn)
    base_acc_ts  = base_model.score(X_ts, y_ts)
    print(base_acc_trn)
    print(base_oob)
    print(base_acc_ts)
    print(confusion_matrix(y_ts, y_pred))
    print(classification_report(y_ts, y_pred))



# Get Data

In [5]:
train=pd.read_csv(f"{PATH}train.csv")
test=pd.read_csv(f"{PATH}test.csv")
train.shape, test.shape

((7613, 5), (3263, 4))

In [9]:
train.target.isnull().sum(), train.fillna("missing", inplace=True)

(0, None)

In [10]:
train.head()

,id,keyword,location,text,target
0,1,missing,missing,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,missing,missing,Forest fire near La Ronge Sask. Canada,1
2,5,missing,missing,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,missing,missing,"13,000 people receive #wildfires evacuation orders in California",1
4,7,missing,missing,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


# Train/Test Split

In [19]:
le_kw = preprocessing.LabelEncoder()
train.keyword=le_kw.fit_transform(train.keyword)

le_loc = preprocessing.LabelEncoder()
train.location=le_loc.fit_transform(train.location)

train.head(2)

,id,keyword,location,text,target
0,1,145,3119,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,145,3119,Forest fire near La Ronge Sask. Canada,1


In [20]:
X_trn, X_ts, y_trn, y_ts = train_test_split(train[["keyword", "location"]], train.target, test_size=0.33, random_state=42)
len(X_trn), y_trn.shape, len(X_ts), y_ts.shape
X_trn.shape, y_trn.shape, X_ts.shape, y_ts.shape


((5100, 2), (5100,), (2513, 2), (2513,))

In [21]:
base_model = RandomForestClassifier(random_state = 42, oob_score=True)
base_model.fit(X_trn, y_trn)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=True, random_state=42, verbose=0, warm_start=False)

In [22]:
evaluate()

0.6586274509803921
0.6629526462395543


In [23]:
y_pred = clf_base.predict(X_ts)
print(confusion_matrix(y_ts, y_pred))
print(classification_report(y_ts, y_pred))

[[1085  361]
 [ 486  581]]
              precision    recall  f1-score   support

           0       0.69      0.75      0.72      1446
           1       0.62      0.54      0.58      1067

   micro avg       0.66      0.66      0.66      2513
   macro avg       0.65      0.65      0.65      2513
weighted avg       0.66      0.66      0.66      2513



# Optimization

## 1. Random Search

In [25]:

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [ ]:
# Use the random grid to search for best hyperparameters
rf = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf_random.fit(X_trn, y_trn)



Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  8.1min


In [ ]:
evaluate(X_trn, y_trn,X_ts, y_ts, rf_random.best_estimator_)

## 2. Grid Search

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [15, 20,25],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [2, 4, 8],
    'n_estimators': [1600, 1800, 2000]
}
# Create a based model
rf = RandomForestClassifier()

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, 
                           param_grid = param_grid, 
                           cv = 3, 
                           n_jobs = -1, 
                           verbose = 2)


# Fit the grid search to the data
grid_search.fit(X_trn, y_trn)

In [ ]:
evaluate(X_trn, y_trn,X_ts, y_ts, grid_search.best_estimator_)